In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
from sklearn.impute import SimpleImputer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_test = pd.read_csv("../input/test.csv")
df_train = pd.read_csv("../input/train.csv")
df_train.head()

In [ ]:
#Converts Embarked location to numbers to enable correlation calcluation
df_test["Embarked"] = df_test["Embarked"].replace("C",1)
df_test["Embarked"] = df_test["Embarked"].replace("Q",2)
df_test["Embarked"] = df_test["Embarked"].replace("S",3)

df_train["Embarked"] = df_train["Embarked"].replace("C",1)
df_train["Embarked"] = df_train["Embarked"].replace("Q",2)
df_train["Embarked"] = df_train["Embarked"].replace("S",3)
df_train.head()

In [ ]:
df_test["Sex"] = df_test["Sex"].replace("male",1)
df_test["Sex"] = df_test["Sex"].replace("female",0)

df_train["Sex"] = df_train["Sex"].replace("male",1)
df_train["Sex"] = df_train["Sex"].replace("female",0)
df_train.head()

In [ ]:
df_train_num = df_train.iloc[:,1:3]
df_train_num["Sex"] = df_train["Sex"] 
df_train_num["SibSp"] = df_train["SibSp"] 
df_train_num["Parch"] = df_train["Parch"] 
df_train_num["Fare"] = df_train["Fare"] 
df_train_num["Embarked"] = df_train["Embarked"] 
df_train_num =df_train_num.reindex(["Pclass","Sex","SibSp","Parch","Fare","Embarked","Survived"], axis=1)
df_train_num.head()

In [ ]:
df_tests = df_test.iloc[:,0:2]
df_tests["Pclass"] = df_test["Pclass"]
df_tests["Sex"] = df_test["Sex"]
df_tests["SibSp"] = df_test["SibSp"]
df_tests["Parch"] = df_test["Parch"]
df_tests["Fare"] = df_test["Fare"]
df_tests["Embarked"] = df_test["Embarked"]
df_tests = df_tests.drop(["PassengerId"], axis=1)

df_tests.head()

In [ ]:
X = df_train_num.drop(["Survived"], axis=1)
Y = df_train["Survived"]

In [ ]:
#Uses simple imputation to populate missing values

#my_imputer = SimpleImputer()
#X = my_imputer.fit_transform(X)

#my_imputer_test = SimpleImputer()
#df_tests = my_imputer.fit_transform(df_tests)
#df_submission = df_test.iloc[:,:]

X.head()

In [ ]:
#Uses fancy imputation to populate missing values

# make copy to avoid changing original data (when Imputing)
new_data = X.copy()
new_data = pd.DataFrame(new_data)
# make new columns indicating what will be imputed
cols_with_missing = (col for col in new_data.columns 
                                 if new_data[col].isnull().any())
for col in cols_with_missing:
    new_data[col + '_was_missing'] = new_data[col].isnull()

new_data.head()

In [ ]:
# Imputation
my_imputer = SimpleImputer()
new_data = pd.DataFrame(my_imputer.fit_transform(new_data))

new_data.head()

In [ ]:
new_data = new_data.drop(6, axis=1)
new_data.columns = pd.DataFrame(X).columns

X = new_data

In [ ]:
# make copy to avoid changing original data (when Imputing)
new_data_test = df_tests.copy()
new_data_test = pd.DataFrame(new_data_test)
# make new columns indicating what will be imputed
cols_with_missing_test = (col for col in new_data_test.columns 
                                 if new_data_test[col].isnull().any())
for col_test in cols_with_missing_test:
    new_data_test[col_test + '_was_missing'] = new_data_test[col_test].isnull()

new_data_test.head()

In [ ]:
# Imputation
my_imputer = SimpleImputer()
new_data_test = pd.DataFrame(my_imputer.fit_transform(new_data_test))

new_data_test = new_data_test.drop(6,axis=1)
new_data_test.columns = pd.DataFrame(df_tests).columns

df_tests = new_data_test

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [ ]:
col_names =  ['PassengerId', 'Survived']
df_submission  = pd.DataFrame(columns = col_names)
df_submission.head()

In [ ]:
df_submission["Survived"] = clf.predict(df_tests)
df_submission["PassengerId"] = df_test["PassengerId"]
df_submission.head()

In [ ]:
#df_submission = df_submission.drop(["Pclass"], axis=1)
#df_submission = df_submission.drop(["Name"], axis=1)
#df_submission = df_submission.drop(["Sex"], axis=1)
#df_submission = df_submission.drop(["Age"], axis=1)
#df_submission = df_submission.drop(["SibSp"], axis=1)
#df_submission = df_submission.drop(["Parch"], axis=1)
#df_submission = df_submission.drop(["Ticket"], axis=1)
#df_submission = df_submission.drop(["Fare"], axis=1)
#df_submission = df_submission.drop(["Cabin"], axis=1)
#df_submission = df_submission.drop(["Embarked"], axis=1)
df_submission.head()

In [ ]:
df_submission.to_csv("AS_Submission_5", index=False)